In [3]:
import pandas as pd
import geopandas as gpd
import fiona
from shapely.geometry import Point, Polygon
from typing import Union, Tuple
from util_detroit import nearest_neighbor, csv_with_x_y_to_gpd

import re,os
import numpy as np
import matplotlib.pyplot as plt

import kml2geojson
import json

pd.options.display.max_columns = None

### S3 data pull

In [154]:
bucket = "s3://secondmeasure-dev-at-foundational-data/hakso/detroit/"
greenlight_fn = f"Project_Green_Light_Locations.geojson"
fn_911 = f"911_Calls_For_Service.csv"
fullpath_911 = f"{bucket}{fn_911}"
fullpath_greenlight = f"{bucket}{greenlight_fn}"
nrows = 1e5
calls = (
    csv_with_x_y_to_gpd(
        #     fullpath,#from s3
        fn_911,  # local
        read_csv_args={
            "nrows": nrows,
            "usecols": [
                "X",
                "Y",
                "incident_id",
                "calldescription",
                "category",
                "call_timestamp",
            ],
        },
    )
    .astype({"call_timestamp": "datetime64"})
    .assign(call_day=lambda df: df.call_timestamp.round("D"))
)

In [98]:
greenlight_geo_df = (
    gpd.read_file(greenlight_fn)
    .astype({"live_date": "datetime64"})
    .assign(live_day=lambda df: df.live_date.round("D"))
)

#### Distance calculation

In [21]:
print(calls.columns)
greenlight_geo_df.columns

Index(['X', 'Y', 'incident_id', 'calldescription', 'category',
       'call_timestamp', 'geometry'],
      dtype='object')


Index(['address', 'business_name', 'business_type', 'precinct', 'live_date',
       'ObjectId', 'geometry'],
      dtype='object')

In [133]:
from util_detroit import nearest_neighbor

In [ ]:
# to do: solve the problem of filtering to only live cameras by selected the top N matches, storing the index and the values in an array in the columns that 
# currently hold the scalars, then write a function to big the shortcut distance for active green lights


# to do: rather than getting closest camera then asking if it's live, instead get all cameras within threshold distance, and ask when the first one went live.

In [ ]:
gre

In [155]:
# greenlight_geo_df = nearest_neighbor(greenlight_geo_df, greenlight_geo_df, 2, True)
PROXIMITY_THRESHOLD = 50
combined_df = nearest_neighbor(
    calls.loc[
        :, ["geometry", "calldescription", "category", "call_timestamp", "call_day"]
    ],
    greenlight_geo_df.loc[:, ["live_date", "live_day", "geometry","address"]],
    50,
).dropna(subset=['date_first_live_camera']).assign(
    days_since_live=lambda df: (df.call_day - df.date_first_live_camera).dt.days,
)
#     within_proximity_of_greenlight=lambda df: df.meters_to_nearest_match
#     <= PROXIMITY_THRESHOLD,
#     live_at_time_of_call=lambda df: df.live_date_neighbor <= df.call_timestamp,
#     close_to_live_light=lambda df: df.live_at_time_of_call
#     & df.within_proximity_of_greenlight,
# )

In [158]:
combined_df

,geometry,calldescription,category,call_timestamp,call_day,date_first_live_camera
20,POINT (-83.22770 42.37154),TRANSPORT PRISONER,TRPPRIS,2016-09-20 10:34:11,2016-09-20,2017-02-14
27,POINT (-83.22770 42.37154),TRANSPORT PRISONER,TRPPRIS,2016-09-20 10:56:15,2016-09-20,2017-02-14
48,POINT (-83.00914 42.37465),RECOVER AUTO,RECAUTO,2016-09-20 11:37:55,2016-09-20,2018-05-30
69,POINT (-82.95726 42.42911),MISCELLANEOUS TRAFFIC,MISCTRAF,2016-09-20 12:03:38,2016-09-21,2017-01-19
100,POINT (-82.97079 42.44557),DISTURBANCE,DISTURB,2016-09-20 12:48:14,2016-09-21,2018-08-24
...,...,...,...,...,...,...
99954,POINT (-83.02423 42.43322),SPECIAL ATTENTION,SPCL ATT,2016-12-11 00:24:45,2016-12-11,2019-12-12
99961,POINT (-83.23596 42.35758),FELONIOUS ASSAULT IP,FA IP,2016-12-11 00:32:19,2016-12-11,2016-01-01
99970,POINT (-83.14786 42.37233),VERIFIED ALR / PERSON W/O CODE,VERALRM,2016-12-11 00:42:15,2016-12-11,2021-06-10
99973,POINT (-83.09584 42.32028),SPECIAL ATTENTION,SPCL ATT,2016-12-11 00:46:03,2016-12-11,2020-10-09


In [114]:
combined_df.columns

Index(['geometry', 'calldescription', 'category', 'call_timestamp', 'call_day',
       'live_date_neighbor', 'live_day_neighbor', 'address_neighbor',
       'meters_to_nearest_match', 'days_since_live',
       'within_proximity_of_greenlight', 'live_at_time_of_call',
       'close_to_live_light'],
      dtype='object')

In [115]:
combined_df.groupby(['address_neighbor','call_day']).category.count()

address_neighbor  call_day  
1 E McNichols Rd  2016-09-23    2
                  2016-09-24    1
                  2016-09-25    1
                  2016-09-26    2
                  2016-09-28    2
                               ..
9970 Freeland St  2016-12-03    1
                  2016-12-04    3
                  2016-12-08    2
                  2016-12-09    4
                  2016-12-10    1
Name: category, Length: 35442, dtype: int64

to do: assume live date is not correlated with crime in proximity of the camera, calculate average proportion of total crime by project greenlight site.

In [123]:
combined_df.loc[lambda df: df.within_proximity_of_greenlight,:].groupby(['address_neighbor','live_at_time_of_call']).category.count().to_frame('n_sites')

n_sites
address_neighbor      live_at_time_of_call         
1 E McNichols Rd      False                      42
1 East State Fair Ave False                      92
1 Park Ave            False                      18
1000 Whitmore Rd      False                      27
10000 Plymouth Rd     False                      32
...                                             ...
9768 Grand River      False                     168
9939 E Jefferson Ave  False                       6
                      True                        3
9949 E Warren Ave     False                      21
9963 Gratiot Ave      False                      30

[460 rows x 1 columns]

In [93]:
background_rate = combined_df.loc[lambda df: df.within_proximity_of_greenlight, :].groupby('call_day').category.count().to_frame('total_calls')

In [122]:
combined_df.loc[lambda df: df.within_proximity_of_greenlight, :].groupby(
    ["live_at_time_of_call", "call_day"]
).category.count().to_frame("grouped_count").join(
    background_rate, on="call_day"
).assign(
    proportion_of_calls=lambda df: df.grouped_count / df.total_calls
).join(
    combined_df.loc[lambda df: df.within_proximity_of_greenlight, :]
    .groupby(["address_neighbor", "live_at_time_of_call"])
    .category.count()
    .to_frame("n_sites")
).assign(proportion_of_calls_per_site=lambda df: df.proportion_of_calls/df.n_sites)

grouped_count  \
live_at_time_of_call call_day   address_neighbor                       
False                2016-09-20 1 E McNichols Rd                   3   
                                1 East State Fair Ave              3   
                                1 Park Ave                         3   
                                1000 Whitmore Rd                   3   
                                10000 Plymouth Rd                  3   
...                                                              ...   
True                 2016-12-11 8536 Linwood Ave                  20   
                                8825 E Jefferson Ave              20   
                                8930 W McNichols Rd               20   
                                975 Mack Ave                      20   
                                9939 E Jefferson Ave              20   

                                                       total_calls  \
live_at_time_of_call call_day   address_neighbor                     
False                2016-09-20 1 E McNichols Rd                 3   
                                1 East State Fair Ave            3   
                                1 Park Ave                       3   
                                1000 Whitmore Rd                 3   
                                10000 Plymouth Rd                3   
...                                                            ...   
True                 2016-12-11 8536 Linwood Ave                69   
                                8825 E Jefferson Ave            69   
                                8930 W McNichols Rd             69   
                                975 Mack Ave                    69   
                                9939 E Jefferson Ave            69   

                                                       proportion_of_calls  \
live_at_time_of_call call_day   address_neighbor                             
False                2016-09-20 1 E McNichols Rd                  1.000000   
                                1 East State Fair Ave             1.000000   
                                1 Park Ave                        1.000000   
                                1000 Whitmore Rd                  1.000000   
                                10000 Plymouth Rd                 1.000000   
...                                                                    ...   
True                 2016-12-11 8536 Linwood Ave                  0.289855   
                                8825 E Jefferson Ave              0.289855   
                                8930 W McNichols Rd               0.289855   
                                975 Mack Ave                      0.289855   
                                9939 E Jefferson Ave              0.289855   

                                                       n_sites  
live_at_time_of_call call_day   address_neighbor                
False                2016-09-20 1 E McNichols Rd            42  
                                1 East State Fair Ave       92  
                                1 Park Ave                  18  
                                1000 Whitmore Rd            27  
                                10000 Plymouth Rd           32  
...                                                        ...  
True                 2016-12-11 8536 Linwood Ave            22  
                                8825 E Jefferson Ave        46  
                                8930 W McNichols Rd         63  
                                975 Mack Ave               221  
                                9939 E Jefferson Ave         3  

[38134 rows x 4 columns]

In [73]:
background_rate

call_day
2016-09-20      67
2016-09-21    1210
2016-09-22    1185
2016-09-23    1167
2016-09-24    1241
              ... 
2016-12-07    1410
2016-12-08    1611
2016-12-09    1474
2016-12-10    1611
2016-12-11     850
Name: incident_id, Length: 83, dtype: int64

## Approximating the relationship between lat/long distance and meters using google maps in detroit


In [15]:
a = (42.450528328751126, -82.94340486981733)
b = (42.440174033911795, -83.27726268725615)
d = 27500

In [18]:
27500/(((a[1]-b[1])**2)-((a[0]-b[0])**2))**.5

82410.03766311899